# Data Cleaning
Youssef Nakhla 900201430
Karim AbouDaoud 900212779


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats
import plotly.figure_factory as ff
import seaborn as sns
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv("hotel_bookings.csv")
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01/07/2015
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01/07/2015
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,02/07/2015
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,02/07/2015
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,03/07/2015


# Checking For NA's

In [3]:
df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [4]:
#dropping 4 NAs of variable children
df.dropna(subset=["children"],inplace=True) 

In [5]:
#94% of company is missing so it will be dropped
df=df.drop(["company"],axis=1) 

In [6]:
df=df.drop(["agent"],axis=1) 

In [7]:
df["country"].value_counts()

PRT    48586
GBR    12129
FRA    10415
ESP     8568
DEU     7287
       ...  
DJI        1
BWA        1
HND        1
VGB        1
NAM        1
Name: country, Length: 177, dtype: int64

In [8]:
df["country"].isnull().sum()

488

In [9]:
df["country"].fillna(df["country"].mode()[0], inplace=True) #filling NAs with the most frequent values

In [10]:
df.isnull().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests   

# Hotel

In [11]:
class_names = df["hotel"].unique()
class_names

array(['Resort Hotel', 'City Hotel'], dtype=object)

In [12]:
df.loc[ df["hotel"] == class_names[0],"hotel"]=0 #resort =0
df.loc[ df["hotel"] == class_names[1],"hotel"]=1 #city =1
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,0,342,2015,July,27,1,0,0,2,...,C,3,No Deposit,0,Transient,0.00,0,0,Check-Out,01/07/2015
1,0,0,737,2015,July,27,1,0,0,2,...,C,4,No Deposit,0,Transient,0.00,0,0,Check-Out,01/07/2015
2,0,0,7,2015,July,27,1,0,1,1,...,C,0,No Deposit,0,Transient,75.00,0,0,Check-Out,02/07/2015
3,0,0,13,2015,July,27,1,0,1,1,...,A,0,No Deposit,0,Transient,75.00,0,0,Check-Out,02/07/2015
4,0,0,14,2015,July,27,1,0,2,2,...,A,0,No Deposit,0,Transient,98.00,0,1,Check-Out,03/07/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,1,0,23,2017,August,35,30,2,5,2,...,A,0,No Deposit,0,Transient,96.14,0,0,Check-Out,06/09/2017
119386,1,0,102,2017,August,35,31,2,5,3,...,E,0,No Deposit,0,Transient,225.43,0,2,Check-Out,07/09/2017
119387,1,0,34,2017,August,35,31,2,5,2,...,D,0,No Deposit,0,Transient,157.71,0,4,Check-Out,07/09/2017
119388,1,0,109,2017,August,35,31,2,5,2,...,A,0,No Deposit,0,Transient,104.40,0,0,Check-Out,07/09/2017


# Lead Time

In [13]:
# Create a MinMaxScaler
scaler = MinMaxScaler()

# Reshape the 'lead_time' column because fit_transform expects a 2D array
df['lead_time_normalized'] = scaler.fit_transform(df['lead_time'].values.reshape(-1, 1))

# Display the first few rows of the DataFrame with the normalized column
print(df[['lead_time', 'lead_time_normalized']].head())


   lead_time  lead_time_normalized
0        342              0.464043
1        737              1.000000
2          7              0.009498
3         13              0.017639
4         14              0.018996


In [14]:
df=df.drop(["lead_time"],axis=1) 

# Arrival Date (Year, Month, Day of Month)

In [15]:
class_names=df["arrival_date_month"].unique()
class_names

array(['July', 'August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June'],
      dtype=object)

In [16]:
df.loc[ df["arrival_date_month"] == class_names[0],"arrival_date_month"]=7 
df.loc[ df["arrival_date_month"] == class_names[1],"arrival_date_month"]=8
df.loc[ df["arrival_date_month"] == class_names[2],"arrival_date_month"]=9
df.loc[ df["arrival_date_month"] == class_names[3],"arrival_date_month"]=10
df.loc[ df["arrival_date_month"] == class_names[4],"arrival_date_month"]=11
df.loc[ df["arrival_date_month"] == class_names[5],"arrival_date_month"]=12
df.loc[ df["arrival_date_month"] == class_names[6],"arrival_date_month"]=1
df.loc[ df["arrival_date_month"] == class_names[7],"arrival_date_month"]=2
df.loc[ df["arrival_date_month"] == class_names[8],"arrival_date_month"]=3
df.loc[ df["arrival_date_month"] == class_names[9],"arrival_date_month"]=4 
df.loc[ df["arrival_date_month"] == class_names[10],"arrival_date_month"]=5
df.loc[ df["arrival_date_month"] == class_names[11],"arrival_date_month"]=6 
df["arrival_date_month"] = df["arrival_date_month"].astype(int)


In [17]:
df.corr(method='pearson')['is_canceled']["arrival_date_day_of_month"]


-0.006084131239587312

In [18]:
df.corr(method='pearson')['is_canceled']["arrival_date_week_number"]

0.008131990729401564

In [19]:
df['arrival_date'] = df['arrival_date_year'].astype(str)+'-'+df['arrival_date_month'].astype(str)+'-'+df['arrival_date_day_of_month'].astype(str)

In [20]:
df['arrival_date']

0          2015-7-1
1          2015-7-1
2          2015-7-1
3          2015-7-1
4          2015-7-1
            ...    
119385    2017-8-30
119386    2017-8-31
119387    2017-8-31
119388    2017-8-31
119389    2017-8-29
Name: arrival_date, Length: 119386, dtype: object

In [21]:
df=df.drop(["arrival_date_year"],axis=1) 

In [22]:
df=df.drop(["arrival_date_month"],axis=1) 

In [23]:
df=df.drop(["arrival_date_day_of_month"],axis=1) 

In [24]:
df=df.drop(["arrival_date_week_number"],axis=1) 

# Weekdays & Week nights

In [25]:
df["total_stays"]=df["stays_in_weekend_nights"]+df["stays_in_week_nights"]

In [26]:
df=df.drop(["stays_in_week_nights"],axis=1) 

In [27]:
df=df.drop(["stays_in_weekend_nights"],axis=1) 

# Adults 

In [28]:
df.drop(df[df["adults"]==0].index,inplace=True)

# Children & Babies

In [29]:
df['kids'] = df['children'] + df['babies']
df["kids"] = df["kids"].astype(int)

In [30]:
df=df.drop(["babies"],axis=1) 

In [31]:
df=df.drop(["children"],axis=1) 

# Meal Type

In [32]:
df=df.replace("Undefined","SC")

In [33]:

df["BB"] = df['meal'].apply(lambda x: 1 if x == 'BB' else 0) 

df["FB"] = df['meal'].apply(lambda x: 1 if x == 'FB' else 0)

df["HB"] = df['meal'].apply(lambda x: 1 if x == 'HB' else 0)

df["SC"] = df['meal'].apply(lambda x: 1 if x == 'SC' else 0)

In [34]:
df=df.drop(["meal"],axis=1) 

# Country

In [35]:
df["Portugal"] = df['country'].apply(lambda x: 1 if x == 'PRT' else 0) #the hotels in this data set are located in Portugal

df["International"]=df['country'].apply(lambda x: 0 if x == 'PRT' else 1)

In [36]:
df=df.drop(["country"],axis=1) 

# Market Segment

In [37]:
unique_values = df['market_segment'].unique()
unique_values

array(['Direct', 'Corporate', 'Online TA', 'Offline TA/TO',
       'Complementary', 'Groups', 'Aviation'], dtype=object)

In [38]:

df["Direct"] = df['market_segment'].apply(lambda x: 1 if x == 'Direct' else 0) 

df["Corporate"] = df['market_segment'].apply(lambda x: 1 if x == 'Corporate' else 0)

df["Online TA"] = df['market_segment'].apply(lambda x: 1 if x == 'Online TA' else 0)

df["Offline TA/TO"] = df['market_segment'].apply(lambda x: 1 if x == 'Offline TA/TO' else 0)

df["Complementary"] = df['market_segment'].apply(lambda x: 1 if x == 'Complementary' else 0)

df["Groups"] = df['market_segment'].apply(lambda x: 1 if x == 'Groups' else 0)

df["Aviation"] = df['market_segment'].apply(lambda x: 1 if x == 'Aviation' else 0)

In [39]:
df=df.drop(["market_segment"],axis=1) 

# Distribution Channel

In [40]:
unique_values = df['distribution_channel'].unique()
unique_values

array(['Direct', 'Corporate', 'TA/TO', 'SC', 'GDS'], dtype=object)

In [41]:
df["Dist Direct"] = df['distribution_channel'].apply(lambda x: 1 if x == 'Direct' else 0)

df["Dist Corporate"] = df['distribution_channel'].apply(lambda x: 1 if x == 'Corporate' else 0)

df["Dist TA/TO"] = df['distribution_channel'].apply(lambda x: 1 if x == 'TA/TO' else 0)

df["SC"] = df['distribution_channel'].apply(lambda x: 1 if x == 'SC' else 0)

df["GDS"] = df['distribution_channel'].apply(lambda x: 1 if x == 'GDS' else 0)

In [42]:
df=df.drop(["distribution_channel"],axis=1) 

# Reserved & Assigned Room Type

In [43]:
class_names = df['assigned_room_type'].unique()

In [44]:
for i in range(len(class_names)):
    df.loc[ df['assigned_room_type'] == class_names[i],'assigned_room_type']=i

In [45]:
class_names2 = df['reserved_room_type'].unique()

In [46]:
for i in range(len(class_names2)):
    df.loc[ df['reserved_room_type'] == class_names2[i],'reserved_room_type']=i

# Deposit Type

In [47]:
unique_values = df['deposit_type'].unique()
unique_values

array(['No Deposit', 'Refundable', 'Non Refund'], dtype=object)

In [48]:
df["No Deposit"] = df['deposit_type'].apply(lambda x: 1 if x == 'No Deposit' else 0)

df["Refundable"] = df['deposit_type'].apply(lambda x: 1 if x == 'Refundable' else 0)

df["Non Refund"] = df['deposit_type'].apply(lambda x: 1 if x == 'Non Refund' else 0)

In [49]:
df=df.drop(["deposit_type"],axis=1) 

# Customer Type

In [50]:
unique_values = df['customer_type'].unique()
unique_values

array(['Transient', 'Contract', 'Transient-Party', 'Group'], dtype=object)

In [51]:
df["Transient"] = df['customer_type'].apply(lambda x: 1 if x == 'Transient' else 0)

df["Contract"] = df['customer_type'].apply(lambda x: 1 if x == 'Contract' else 0)

df["Transient-Party"] = df['customer_type'].apply(lambda x: 1 if x == 'Transient-Party' else 0)

df["Group"] = df['customer_type'].apply(lambda x: 1 if x == 'Group' else 0)

In [52]:
df=df.drop(["customer_type"],axis=1) 

# ADR

In [53]:
df[df["adr"]>500]

,hotel,is_canceled,adults,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,days_in_waiting_list,...,Dist Corporate,Dist TA/TO,GDS,No Deposit,Refundable,Non Refund,Transient,Contract,Transient-Party,Group
15083,0,0,2,1,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,0
48515,1,1,2,0,0,0,1,1,1,0,...,0,1,0,0,0,1,1,0,0,0
111403,1,0,1,0,0,0,1,4,0,0,...,0,1,0,1,0,0,1,0,0,0


In [54]:
df["adr"]=df["adr"].replace(5400.0,540)

# Reservation Status

In [55]:
class_names = df['reservation_status'].unique()
class_names

array(['Check-Out', 'Canceled', 'No-Show'], dtype=object)

In [56]:
for i in range(len(class_names)):
    df.loc[ df['reservation_status'] == class_names[i],'reservation_status']=i

#mapping the "No-Show" varaible with the "Canceled" variable by setting its label to 1
df["reservation_status"]=df["reservation_status"].replace(2,1)

In [57]:
df=df.drop(["reservation_status"],axis=1) 

# Reservation Date

In [58]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'], format='%d/%m/%Y')
df['reservation_status_date']

0        2015-07-01
1        2015-07-01
2        2015-07-02
3        2015-07-02
4        2015-07-03
            ...    
119385   2017-09-06
119386   2017-09-07
119387   2017-09-07
119388   2017-09-07
119389   2017-09-07
Name: reservation_status_date, Length: 118983, dtype: datetime64[ns]

# New Feature "Total Guests"

In [59]:
df["total_guests"]=df["adults"]+df["kids"]

# Post Data Cleaning

In [60]:
df.columns

Index(['hotel', 'is_canceled', 'adults', 'is_repeated_guest',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'assigned_room_type', 'booking_changes',
       'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests', 'reservation_status_date',
       'lead_time_normalized', 'arrival_date', 'total_stays', 'kids', 'BB',
       'FB', 'HB', 'SC', 'Portugal', 'International', 'Direct', 'Corporate',
       'Online TA', 'Offline TA/TO', 'Complementary', 'Groups', 'Aviation',
       'Dist Direct', 'Dist Corporate', 'Dist TA/TO', 'GDS', 'No Deposit',
       'Refundable', 'Non Refund', 'Transient', 'Contract', 'Transient-Party',
       'Group', 'total_guests'],
      dtype='object')

In [61]:
num_columns = df.shape[1]
num_columns

43

In [62]:
num_observations = len(df)
num_observations

118983

In [63]:

df.to_csv('Post-cleaning Data.csv', index=False, mode='w')


In [64]:
df.head()

,hotel,is_canceled,adults,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,days_in_waiting_list,...,Dist TA/TO,GDS,No Deposit,Refundable,Non Refund,Transient,Contract,Transient-Party,Group,total_guests
0,0,0,2,0,0,0,0,0,3,0,...,0,0,1,0,0,1,0,0,0,2
1,0,0,2,0,0,0,0,0,4,0,...,0,0,1,0,0,1,0,0,0,2
2,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1
3,0,0,1,0,0,0,1,1,0,0,...,0,0,1,0,0,1,0,0,0,1
4,0,0,2,0,0,0,1,1,0,0,...,1,0,1,0,0,1,0,0,0,2
